# Chance-constrained programming: portfolio optimization example

In [18]:
using LinearAlgebra
using JuMP
using Gurobi
using Distributions
using Random

Adapted from Stephen Boyd and Lieven Vandenberghe, "Convex Optimization", Cambridge University Press, 2004, Section 4.7.6, p. 187.

We consider an investiment portfolio with $n$ assets and random returns, except the last one. We assume that the $n-1$ first returns follow a multivariate distribution with known mean and covariance matrix. The last asset is a risk-free investment product, with a fixed return.

The assets characteristics are detailed below.

In [19]:
n = 4  # the last asset is a risk-free asset, with a null variance.
μ = [.12 ; .10 ; .07 ; .03]
Σ = [ 4e-2  6e-3 -4e-3  ;
      6e-3  1e-2  0.0  ;
      -4e-3  0.0 2.5e-3 ]

# We create une multivariate normal of mean μ and covariance matrix Σ
d = MvNormal(μ[1:n-1], Σ)

FullNormal(
dim: 3
μ: [0.12, 0.1, 0.07]
Σ: [0.04 0.006 -0.004; 0.006 0.01 0.0; -0.004 0.0 0.0025]
)


In [20]:
# Estimate the probability to have a negative return and the resulting expected shortfall.
function expectedshortfall(p:: Vector, d:: Distribution, M:: Int = 1000000)
    
    loss = 0
    vloss = 0
    for i = 1:M
        ξ = [rand(d); μ[n]]
        ret = dot(p, ξ)
        if ret < 0
            loss += 1
            vloss += ret
        end
    end

    return loss/M, vloss/loss
    
end

expectedshortfall (generic function with 2 methods)

## Portfolio with uniform repartition

We first consider the naive strategy where the same amount is invested in each asset.

In [22]:
# Expected return with a uniform repartition.
p = ones(n)./n
er = sum(p[i]*μ[i] for i = 1:n)

0.08000000000000002

We now compute the loss probability and the average loss when a loss occurs.

In [23]:
expectedshortfall(p, d)

(0.089097, -0.02749181127080138)

## Optimal decision without loss constraint

We now aim to maximize the expected return, without any consideration for the potential loss.

In [24]:
m = Model(with_optimizer(Gurobi.Optimizer))

@variable(m, p[1:n] >= -0.1)
@constraint(m, sum(p[i] for i = 1:n) <= 1)

@objective(m, Max, sum(p[i]*μ[i] for i = 1:n))

println(m)

Academic license - for non-commercial use only - expires 2021-10-29
Max 0.12 p[1] + 0.1 p[2] + 0.07 p[3] + 0.03 p[4]
Subject to
 p[1] + p[2] + p[3] + p[4] <= 1.0
 p[1] >= -0.1
 p[2] >= -0.1
 p[3] >= -0.1
 p[4] >= -0.1



In [25]:
optimize!(m)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xcc387da0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 1e-01]
  Bounds range     [1e-01, 1e-01]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3600000e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  1.360000000e-01

User-callback calls 26, time in user-callback 0.00 sec


In [26]:
value.(p)

4-element Vector{Float64}:
  1.3000000000000003
 -0.1
 -0.1
 -0.1

Not surprisingly, we invest everything is the asset having the highest return rate, even borrowing from the lower return rate asset.

In [27]:
objective_value(m)

0.136

The loss probability is however close to 30%, and the average lost amount is significantly more important if a loss occurs.

In [28]:
expectedshortfall(value.(p), d)

(0.299643, -0.16430793157252646)

## Optimal decision with loss constraint

We add the constraint that we want to limit the risk by accepting a loss with a maximum probability of 0.05.

In [29]:
maxloss = 0

α = 0.95
z = 1/quantile(Normal(0,1), α)

0.6079568319117693

We build the second-order cone constraint corresponding to the joint chance constraint.

In [30]:
A = Σ^0.5

3×3 Symmetric{Float64, Matrix{Float64}}:
  0.198273   0.0203895   -0.0164913
  0.0203895  0.0978718    0.00231869
 -0.0164913  0.00231869   0.0471451

In [32]:
A*A-Σ

3×3 Matrix{Float64}:
 -6.93889e-18   5.20417e-18   1.73472e-18
  5.20417e-18  -1.73472e-18  -1.22964e-18
  1.73472e-18  -1.22964e-18   0.0

In [36]:
# || x || <= t, t >= 0
# https://jump.dev/JuMP.jl/stable/reference/constraints/#JuMP.SecondOrderCone
@constraint(m, [z*(-maxloss+sum(μ[i]*p[i] for i = 1:n)); (Σ^0.5)*p[1:n-1]] in SecondOrderCone())

println(m)

Max 0.12 p[1] + 0.1 p[2] + 0.07 p[3] + 0.03 p[4]
Subject to
 p[1] + p[2] + p[3] + p[4] <= 1.0
 [0.0729548198294123 p[1] + 0.06079568319117693 p[2] + 0.04255697823382386 p[3] + 0.018238704957353077 p[4], 0.19827331129330036 p[1] + 0.02038945638178348 p[2] - 0.016491334004420634 p[3], 0.02038945638178348 p[1] + 0.09787182302877327 p[2] + 0.002318690466585327 p[3], -0.016491334004420634 p[1] + 0.002318690466585327 p[2] + 0.04714509070173485 p[3]] in MathOptInterface.SecondOrderCone(4)
 [0.0729548198294123 p[1] + 0.06079568319117693 p[2] + 0.04255697823382386 p[3] + 0.018238704957353077 p[4], 0.19827331129330036 p[1] + 0.02038945638178348 p[2] - 0.016491334004420634 p[3], 0.02038945638178348 p[1] + 0.09787182302877327 p[2] + 0.002318690466585327 p[3], -0.016491334004420634 p[1] + 0.002318690466585327 p[2] + 0.04714509070173485 p[3]] in MathOptInterface.SecondOrderCone(4)
 p[1] >= -0.1
 p[2] >= -0.1
 p[3] >= -0.1
 p[4] >= -0.1



In [37]:
optimize!(m)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 9 rows, 12 columns and 38 nonzeros
Model has 2 quadratic constraints
Coefficient statistics:
  Matrix range     [2e-03, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e-02, 1e-01]
  Bounds range     [1e-01, 1e-01]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 9 rows, 12 columns, 38 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.600e+01
 Factor NZ  : 4.500e+01
 Factor Ops : 2.850e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.04431570e-01  5.62123875e-02  3.16e-01 1.24e-01  2.24e-02     0s
   1   8.92965198e-02  1.01222277e-01  1.93e-02 9.91e-03  3.67e-03     0s
   2   8.54648347e-02  9.8

In [38]:
sol = value.(p)

4-element Vector{Float64}:
  0.18186244541024285
  0.34027374502773744
  0.577863790037057
 -0.0999999915994766

In [39]:
objective_value(m)

0.09330133350661257

Without any surprise, the expected return is less than without the loss constraint, but is still higher than with the uniform repartition. We can also see that we use the risk-free asset as a borrowing tool. The risk to lose money is limited to 5%, as desired, and is less than any other strategy. Interestingly, the average loss is also the smallest one.

In [40]:
expectedshortfall(value.(p), d)

(0.049992, -0.023570703787206635)